# Topological text analysis - similarity of movie descriptions

In this notebook we will present how topological methods can be used to analyse text.

The main idea is to decompose the given text into blocks, creating a corpus, after which we would use the TfIdf method to obtain numerical values i.e. points from blocks of text. These points are later subject to the topological methods: we use Vietrois-Rips filtration to create persistence diagrams of given movie description. Subsequently, we use the Bottleneck distance of persistence diagrams to establish the level of similarity between descriptions.

We start by uploading the necessary python packages:

In [5]:
import numpy as np
import pandas as pd
import os
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import metrics
import math

For TDA, we use the giotto library:

In [8]:
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PairwiseDistance

Now, we upload the necessary packages for text processing:

In [11]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from collections import defaultdict
import re
from langdetect import detect

We are ready to begin.  
The movie of the most interest to us would be the "Harry Potter and the Philospher's Stone".  
Given a file of multiple movie descriptions, we would like to see what would be the movies most similar to the given one, based on their descriptions, using the topological methods.

In [27]:
main_description = ''
with open("Harry Potter description.txt","r") as f:
    main_description = f.read()

In [29]:
print(main_description)

Ten years later, just before Harry's eleventh birthday, owls begin delivering letters addressed to him. When the abusive Dursleys adamantly refuse to allow Harry to open any and flee to an island hut, Hagrid arrives to personally deliver Harry's letter of acceptance to Hogwarts. Hagrid also reveals that Harry's late parents, James and Lily, were killed by a dark wizard named Lord Voldemort. The killing curse that Voldemort had cast towards Harry rebounded, destroying Voldemort's body and giving Harry the lightning-bolt scar on his forehead. Hagrid then takes Harry to Diagon Alley for school supplies and gives him a pet snowy owl whom he names Hedwig. Harry buys a wand that is connected to Voldemort's own wand. At King's Cross, Harry boards the Hogwarts Express train, and meets fellow first-years Ron Weasley and Hermione Granger during the journey. Arriving at Hogwarts, Harry also meets Draco Malfoy, who is from a wealthy wizard family; the two immediately form a rivalry. The students a

The movie descriptions are stored in the file "MovieData.csv". We will now read this data, but we will only consider the lengthy descriptions of movies, since the use of topological methods would make sense. Additionally, we will only consider those descriptions that are written in english.

In [19]:
def read_data(limit=1000):
    movies = pd.read_csv('MovieData.csv')
    movies['desc'] = movies['desc'].fillna(' ') # fill the NaN slots with a blank line
    movies = movies[movies['desc'].apply(len) > limit] # we only consider the descriptions longer than *limit* characters
    movies = movies.drop_duplicates(subset = ['title']) # drop descriptions of movies that apear multiple times
    movies['lang'] = movies['desc'].apply(detect) # detect the language in which the descriptions were made
    movies = movies[movies['lang'] == 'en'] # select only the descriptions written in English
    movies = movies.drop('lang', axis=1) # delete the 'lang' column, since we won't be using it in the future

    return movies

In [21]:
movies = read_data()

We can see some samples of given movie descriptions:

In [24]:
print(movies.head())

                 title                                               desc
0      Sorrowful Jones  Sorrowful Jones is a New York bookie who keeps...
1   South of St. Louis  During the Civil War, Kip Davis (Joel McCrea),...
4      Strange Bargain  Because the firm is bankrupt, bookkeeper Sam W...
6    Streets of Laredo  A trio of outlaws, Jim Dawkins (Holden), Loren...
11          Task Force  As a 1917 graduate of the Naval Academy, Naval...


Wi will now process all of this data:

In [34]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Korisnik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Korisnik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Korisnik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Korisnik\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [36]:
stops = stopwords.words('english')
lemma = WordNetLemmatizer()

In [42]:
tag_map = defaultdict(lambda: 'n')
tag_map['J'] = 'a'
tag_map['V'] = 'v'
tag_map['R'] = 'r'

def get_wordnet_tags(tokens):
    tagged_tokens = pos_tag(tokens)
    return [(token[0], tag_map[token[1][0]]) for token in tagged_tokens]

def process_text(text):
    text = text.lower()
    text = re.sub("[^a-z ]", " ", text)
    tokens = word_tokenize(text)
    tokens = get_wordnet_tags(tokens)
    tokens = [lemma.lemmatize(word=token[0], pos=token[1]) for token in tokens]
    tokens = [word for word in tokens if word not in stops and len(word) > 2]
    return ' '.join(tokens)

def process_data( movies ):
    movies['processed desc'] = movies['desc'].apply(process_text)
    return movies

In [40]:
processed_main_description = process_text(main_description)

print(processed_main_description)

ten year later harry eleventh birthday owls begin deliver letter address abusive dursleys adamantly refuse allow harry open flee island hut hagrid arrive personally deliver harry letter acceptance hogwarts hagrid also reveal harry late parent james lily kill dark wizard name lord voldemort kill curse voldemort cast towards harry rebound destroy voldemort body give harry lightning bolt scar forehead hagrid take harry diagon alley school supply give pet snowy owl name hedwig harry buy wand connect voldemort wand king cross harry board hogwarts express train meet fellow first year ron weasley hermione granger journey arrive hogwarts harry also meet draco malfoy wealthy wizard family two immediately form rivalry student assemble great hall sort hat sort first year four respective house gryffindor hufflepuff ravenclaw slytherin harry place gryffindor alongside ron hermione draco place slytherin house note dark wizard


In [44]:
movies = process_data(movies)

Since we have finished the processing of the text we are ready to use the TDA tools:  
As has been said in the beginning, each description will be decomposed into several blocks of text. Upon applying the TfIdf method, these blocks will be transformed into vectors i.e. points. These points will be subject to the Vietoris-Rips filtration based on which we will otain the persistence classes of every description. Based on the bottleneck distance of persistence diagrams, we will find the most similar movies to the one of "Harry Potter and the Philosopher's Stone".

In [48]:
def block_decomposition(text, number_of_blocks=5):
    text_len = len(text)
    block_len = text_len//number_of_blocks
    blocks = [text[i:i+block_len] for i in range(0, text_len, block_len)]
    if len(blocks)%2 == 1:
        blocks.append('')
    return blocks

def get_points(text, number_of_blocks=5):
    blocks = block_decomposition(text=text, number_of_blocks=number_of_blocks)
    vectorizer = TfidfVectorizer()
    prepoints = vectorizer.fit_transform(blocks)
    pca = PCA(n_components=2)
    points = pca.fit_transform(prepoints.toarray())
    return points

In [50]:
def get_diagram(points, edge_length=0.5):
    vrp = VietorisRipsPersistence(metric='euclidean', homology_dimensions=[0, 1], max_edge_length=edge_length)
    return vrp.fit_transform(points[None,:,:])

In [52]:
def bottlenck_distance(diagram1, diagram2):
    bottleneck_dist = PairwiseDistance(metric='bottleneck')
    bottleneck_dist.fit(diagram1)
    d = bottleneck_dist.transform(diagram2)
    return d[0][0]

We are ready to proceed:

In [55]:
main_points = get_points(processed_main_description)
main_diagram = get_diagram(main_points)

distances = []

for movie_description in movies['processed desc']:
    points = get_points(movie_description)
    diagram = get_diagram(points)
    distances.append(bottlenck_distance(main_diagram, diagram))

After this process is finished we can see what does the topology of given description says which movies are most similar to the given one:

In [59]:
sorted_movies = sorted(zip(movies['title'], movies['desc'], distances), key=lambda x: x[2], reverse=True)
top_10_similar_movies = sorted_movies[:10]
print("Top 10 similar movies are:")

for i in range(10):
    print(str(i+1)+": "+top_10_similar_movies[i][0])
    print(top_10_similar_movies[i][1])
    print('==================================')

Top 10 similar movies are:
1: The First Legion
Fr. John Fulton, a Jesuit instructor in a seminary school, feels he has lost his vocation. A talk with his friend Fr. Marc Arnoux is no help. But on the night he plans to leave the seminary (and the Order) his old teacher Fr. Jose Sierra miraculously gets up and walks, to tell him to stay. The young, wheelchair-bound neighbor Terry Gilmartin regains hope a similar miracle might allow her to walk; her physician, Dr. Peter Morrell, the same one who attended Fr. Sierra, and who is in love with Terry, confesses that he had engineered Sierra's miraculous recovery, to Fr. Arnoux, but refuses his advice to tell the truth. The Jesuit seminary rector orders Fr. Arnoux to plead the validity of the miracle before the Vatican, in Rome. When his highly respected subordinate refuses, the rector dies of a heart attack. At that point Dr. Morrell admits his deception, in particular to Terry, who goes to the seminary chapel and, miraculously, gets out of he

As we can see, the TDA has produced some unexpected results. Surely we have hoped that the other movies of the "Harry Potter" film series (which are present in the file "MovieData.csv") would make the top 10 list.  
Nontheless, we can see that the movies from the list do share some similarities to the description of the first installment of the "Harry Potter" franchise: there are mentions of school, friends, murder etc.  
This does indicate that TDA produces meaningful results. On the other hand, the shortcomings of this particular method can be explained by the relatively small number of points representing each of the descriptions. We can ovecome this problem by simply increasing the number of blocs, but that would produce less valuable TfIdf values. This leads to the conclussion that the algorithm would produce better results if we would have drastically longer descriptions than the ones given, which could be the subject for further research.